# ALG 3 Analysis - All-Pass Filter (MUTED)

## Overview

ALG 3 (22kHz mode) is an **all-pass filter** used by slot 6.
This slot is **MUTED** (mix_l=0, mix_r=0) - it processes audio through SRAM
but contributes nothing to the DAC output directly.

### Key Characteristics
- **A-RAM location:** 0xC0-0xFF (64 instructions)
- **D-RAM ALG:** 6 (maps to 22kHz ALG 3)
- **Used by:** Slot 6 only
- **WACC instructions:** 2 (PC11, PC33) - but produce zero output (muted)
- **WXY+WSP:** 1 at PC05 - updates MIX (to 0/0)
- **Purpose:** All-pass diffusion for reverb feedback network

### Signal Flow
```
SRAM delay buffer -> WXY read -> all-pass processing -> SRAM write-back
                                                      |
                                                      v
                                        Feedback for other slots to read
```

### Why Muted?
The all-pass filter contributes diffuse reverb texture by processing
SRAM buffers that other slots (7-11) read from. It doesn't need to
produce direct output - its contribution is through the shared delay buffers.

In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from sam8905_interpreter import (
    SAM8905Interpreter,
    plot_waveform,
    export_wav,
    print_state,
    print_dram_changes
)
from sam8905_aram_decoder import decode_algorithm

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)

## ALG 3 Algorithm (A-RAM 0xC0-0xFF)

64 instructions for the all-pass filter network.

In [2]:
# ALG 3: All-pass filter (MUTED - processes SRAM for feedback)
# Used by slot 6 only
# Has WXY+WSP at PC05 - but updates MIX to muted values (0,0)

aram_alg3 = [
    # === INITIAL SETUP (PC00-PC04) ===
    0x2ADF,  # PC00: RADD  5, <WM>                  - D[5] = A+B (from prev slot)
    0x10FD,  # PC01: RM    2, <WWF>                 - WWF = D[2]
    0x006F,  # PC02: RM    0, <WA, WPHI>            - A = PHI = D[0]
    0x18BF,  # PC03: RM    3, <WB>                  - B = D[3]
    0x02DF,  # PC04: RADD  0, <WM>                  - D[0] = A+B
    
    # === MIX UPDATE + PROCESSING (PC05-PC11) ===
    0x39F7,  # PC05: RM    7, <WXY> [WSP]           - **WXY+WSP - MIX = muted**
    0x287F,  # PC06: RM    5, <WA>                  - A = D[5]
    0x7CBF,  # PC07: RP   15, <WB>                  - B = product
    0x40F7,  # PC08: RM    8, <WXY>                 - WXY = D[8]
    0x32DF,  # PC09: RADD  6, <WM>                  - D[6] = A+B
    0x7C3F,  # PC10: RP   15, <WA, WB>              - A = B = product
    0x6ADE,  # PC11: RADD 13, <WM, WACC>            - D[13]=A+B, **WACC #1** (muted)
    
    # === SECOND PROCESSING BLOCK (PC12-PC24) ===
    0x006F,  # PC12: RM    0, <WA, WPHI>            - A = PHI = D[0]
    0x20BF,  # PC13: RM    4, <WB>                  - B = D[4]
    0x02DF,  # PC14: RADD  0, <WM>                  - D[0] = A+B
    0x087F,  # PC15: RM    1, <WA>                  - A = D[1]
    0x40F7,  # PC16: RM    8, <WXY>                 - WXY
    0x0ADF,  # PC17: RADD  1, <WM>                  - D[1] = A+B
    0x74DF,  # PC18: RP   14, <WM>                  - D[14] = product
    0x38F7,  # PC19: RM    7, <WXY>                 - WXY
    0x307F,  # PC20: RM    6, <WA>                  - A = D[6]
    0x7CEF,  # PC21: RP   15, <WPHI>                - PHI = product
    0x60FD,  # PC22: RM   12, <WWF>                 - WWF = D[12]
    0x48F7,  # PC23: RM    9, <WXY>                 - WXY
    0x7FFF,  # PC24: RSP  15, [WSP]                 - Sync
    
    # === THIRD PROCESSING BLOCK (PC25-PC41) ===
    0x7CBF,  # PC25: RP   15, <WB>                  - B = product
    0x70EF,  # PC26: RM   14, <WPHI>                - PHI = D[14]
    0x48F7,  # PC27: RM    9, <WXY>                 - WXY
    0x325F,  # PC28: RADD  6, <WA, WM>              - D[6]=A+B, A=bus
    0x68BF,  # PC29: RM   13, <WB>                  - B = D[13]
    0x7A7F,  # PC30: RADD 15, <WA>                  - A = A+B
    0x7CBF,  # PC31: RP   15, <WB>                  - B = product
    0x7A7F,  # PC32: RADD 15, <WA>                  - A = A+B
    0x6ADE,  # PC33: RADD 13, <WM, WACC>            - D[13]=A+B, **WACC #2** (muted)
    0x30F7,  # PC34: RM    6, <WXY>                 - WXY
    0x10FD,  # PC35: RM    2, <WWF>                 - WWF = D[2]
    0x086F,  # PC36: RM    1, <WA, WPHI>            - A = PHI = D[1]
    
    # === SETTLING SEQUENCE (PC37-PC41) ===
    0x7FFB,  # PC37: RSP  15, <clrB> [WSP]          - WWE write
    0x7FFB,  # PC38: RSP  15, <clrB> [WSP]          - WWE write
    0x7EFB,  # PC39: RSP  15, <clrB>                - Settling
    0x7EFB,  # PC40: RSP  15, <clrB>                - Settling
    0x7FFF,  # PC41: RSP  15, [WSP]                 - Sync
    
    # === ACCUMULATION CHAIN (PC42-PC53) ===
    0x18BF,  # PC42: RM    3, <WB>                  - B = D[3]
    0x0ADF,  # PC43: RADD  1, <WM>                  - D[1] = A+B
    0x303F,  # PC44: RM    6, <WA, WB>              - A = B = D[6]
    0x7A3F,  # PC45: RADD 15, <WA, WB>              - Multiply chain
    0x7A3F,  # PC46: RADD 15, <WA, WB>
    0x7A3F,  # PC47: RADD 15, <WA, WB>
    0x7A3F,  # PC48: RADD 15, <WA, WB>
    0x7A3F,  # PC49: RADD 15, <WA, WB>
    0x7A3F,  # PC50: RADD 15, <WA, WB>
    0x7A3F,  # PC51: RADD 15, <WA, WB>
    0x08EF,  # PC52: RM    1, <WPHI>                - PHI = D[1]
    0x7AF7,  # PC53: RADD 15, <WXY>                 - WXY - SRAM read
    
    # === FINAL SETTLING (PC54-PC63) ===
    0x7FFB,  # PC54: RSP  15, <clrB> [WSP]          - WWE write
    0x7FFB,  # PC55: RSP  15, <clrB> [WSP]          - WWE write
    0x7EFB,  # PC56: RSP  15, <clrB>                - Settling
    0x7EFB,  # PC57: RSP  15, <clrB>                - Settling
    0x7FFF,  # PC58: RSP  15, [WSP]                 - Sync
    0x687B,  # PC59: RM   13, <WA, clrB>            - A=D[13], clear B
    0x7FFF,  # PC60: RSP  15, [WSP]                 - Sync
    0x7FFF,  # PC61: RSP  15, [WSP]                 - Sync
    
    # PC62-63: Reserved
    0x7FFF,  # PC62: Reserved
    0x7FFF,  # PC63: Reserved
]

print(f"ALG 3 instructions: {len(aram_alg3)}")
print(decode_algorithm(aram_alg3, alg_num=3))

ALG 3 instructions: 64
=== Algorithm 3 ===

PC00: 2ADF  RADD 5, <WM>
PC01: 10FD  RM 2, <WWF>
PC02: 006F  RM 0, <WA, WPHI>
PC03: 18BF  RM 3, <WB>
PC04: 02DF  RADD 0, <WM>
PC05: 39F7  RM 7, <WXY, WSP> ***
PC06: 287F  RM 5, <WA>
PC07: 7CBF  RP, <WB>
PC08: 40F7  RM 8, <WXY>
PC09: 32DF  RADD 6, <WM>
PC10: 7C3F  RP, <WA, WB>
PC11: 6ADE  RADD 13, <WM, WACC>
PC12: 006F  RM 0, <WA, WPHI>
PC13: 20BF  RM 4, <WB>
PC14: 02DF  RADD 0, <WM>
PC15: 087F  RM 1, <WA>
PC16: 40F7  RM 8, <WXY>
PC17: 0ADF  RADD 1, <WM>
PC18: 74DF  RP 14, <WM>
PC19: 38F7  RM 7, <WXY>
PC20: 307F  RM 6, <WA>
PC21: 7CEF  RP, <WPHI>
PC22: 60FD  RM 12, <WWF>
PC23: 48F7  RM 9, <WXY>
PC24: 7FFF  RSP, <WSP> ***
PC25: 7CBF  RP, <WB>
PC26: 70EF  RM 14, <WPHI>
PC27: 48F7  RM 9, <WXY>
PC28: 325F  RADD 6, <WA, WM>
PC29: 68BF  RM 13, <WB>
PC30: 7A7F  RADD, <WA>
PC31: 7CBF  RP, <WB>
PC32: 7A7F  RADD, <WA>
PC33: 6ADE  RADD 13, <WM, WACC>
PC34: 30F7  RM 6, <WXY>
PC35: 10FD  RM 2, <WWF>
PC36: 086F  RM 1, <WA, WPHI>
PC37: 7FFB  RSP, <clearB, WS

## D-RAM Configuration for Slot 6 (All-Pass Filter)

Note D[7] contains the MIX configuration with mix_l=0, mix_r=0 (muted).

In [3]:
# D-RAM configuration for Slot 6 (all-pass filter, MUTED)
# ALG=6 in D-RAM maps to 22kHz ALG 3 (A-RAM 0xC0-0xFF)

# D[7] contains the MIX configuration for PC05's WXY+WSP
# Check if it's muted (mix_l=0, mix_r=0)
d7_slot6 = 0x52C00
y_coef = (d7_slot6 >> 7) & 0xFFF  # Y coefficient (bits 18:7)
mix_l = (d7_slot6 >> 3) & 0x7     # mix_l (bits 5:3)
mix_r = d7_slot6 & 0x7            # mix_r (bits 2:0)
print(f"D[7] = 0x{d7_slot6:05X}")
print(f"  Y coefficient: 0x{y_coef:03X} ({y_coef})")
print(f"  mix_l: {mix_l} ({['mute','-24dB','-18dB','-12dB','-9dB','-6dB','-3dB','0dB'][mix_l]})")
print(f"  mix_r: {mix_r} ({['mute','-24dB','-18dB','-12dB','-9dB','-6dB','-3dB','0dB'][mix_r]})")

dram_slot6 = [
    0x5F000,  # word 0: Large SRAM address
    0x7FC80,  # word 1: Near-max address
    0x00600,  # word 2: WWF config
    0x00080,  # word 3: Step size
    0x00380,  # word 4: Second step
    0x00000,  # word 5: Working register
    0x00000,  # word 6: Accumulator
    0x52C00,  # word 7: MUTED - mix_l=0, mix_r=0
    0x2D430,  # word 8: SRAM address (comb delay)
    0x00400,  # word 9: Coefficient
    0x7FC00,  # word 10: Large offset
    0x00000,  # word 11: Working
    0x34000,  # word 12: WWF config
    0x00000,  # word 13: Feedback state
    0x00000,  # word 14: Output
    0x00680,  # word 15: IDLE=0, ALG=6 (22kHz ALG 3)
]

print("\nD-RAM slot 6 configuration:")
for i, val in enumerate(dram_slot6):
    signed_val = val if val < 0x40000 else val - 0x80000
    print(f"  D[{i:2d}] = 0x{val:05X} (signed: {signed_val:+d})")

D[7] = 0x52C00
  Y coefficient: 0xA58 (2648)
  mix_l: 0 (mute)
  mix_r: 0 (mute)

D-RAM slot 6 configuration:
  D[ 0] = 0x5F000 (signed: -135168)
  D[ 1] = 0x7FC80 (signed: -896)
  D[ 2] = 0x00600 (signed: +1536)
  D[ 3] = 0x00080 (signed: +128)
  D[ 4] = 0x00380 (signed: +896)
  D[ 5] = 0x00000 (signed: +0)
  D[ 6] = 0x00000 (signed: +0)
  D[ 7] = 0x52C00 (signed: -185344)
  D[ 8] = 0x2D430 (signed: +185392)
  D[ 9] = 0x00400 (signed: +1024)
  D[10] = 0x7FC00 (signed: -1024)
  D[11] = 0x00000 (signed: +0)
  D[12] = 0x34000 (signed: +212992)
  D[13] = 0x00000 (signed: +0)
  D[14] = 0x00000 (signed: +0)
  D[15] = 0x00680 (signed: +1664)


In [4]:
# Initialize SRAM buffer with test pattern
sram_buffer = {}
sram_reads = []
sram_writes = []
frame_counter = 0

# Populate SRAM with decaying sine wave pattern (sparse)
for addr in range(0x00000, 0x100000):
    if addr % 256 == 0:
        decay = max(0, 1.0 - (addr / 0x80000))
        phase = addr * 0.001
        sample = int(decay * 1500 * np.sin(phase))
        sample = max(-2048, min(2047, sample))
        sram_buffer[addr] = sample & 0xFFF

def waveform_read(addr):
    """Callback for external waveform reads."""
    global sram_reads, frame_counter
    nearest_addr = (addr // 256) * 256
    value = sram_buffer.get(nearest_addr, 0)
    sram_reads.append((frame_counter, addr, value))
    if value & 0x800:
        value = value - 0x1000
    return value

def waveform_write(addr, value, phi, pc):
    """Callback for SRAM writes."""
    global sram_writes, frame_counter
    sram_buffer[addr] = value & 0xFFF
    sram_writes.append((frame_counter, addr, value, pc))

print(f"SRAM buffer initialized (sparse, {len(sram_buffer)} locations)")

SRAM buffer initialized (sparse, 4096 locations)


In [5]:
# Create interpreter instance
sam = SAM8905Interpreter()

# Load ALG 3 at A-RAM offset 0xC0 (22kHz ALG 3)
sam.load_aram(aram_alg3, offset=0xC0)

# Load D-RAM for slot 6
sam.load_dram(slot=6, words=dram_slot6)

# Set waveform read callback
sam.waveform_read = waveform_read

# Seed A with value from "previous slot" (as if slot 5 passed data)
sam.state.a = 0x1000

print("Interpreter initialized")
print(f"A-RAM loaded at offset 0xC0 ({len(aram_alg3)} instructions)")
print(f"D-RAM slot 6 loaded ({len(dram_slot6)} words)")
print(f"Initial state: A=0x{sam.state.a:05X}, MIX_L={sam.state.mix_l}, MIX_R={sam.state.mix_r}")

Interpreter initialized
A-RAM loaded at offset 0xC0 (64 instructions)
D-RAM slot 6 loaded (16 words)
Initial state: A=0x01000, MIX_L=0, MIX_R=0


## Execute ALG 3

Run the algorithm. Since it's muted, output should be zero despite internal processing.

In [6]:
# Reset logging
sram_reads = []
sram_writes = []
frame_counter = 0

# Run frame by frame
NUM_FRAMES = 500
samples_list = []

for i in range(NUM_FRAMES):
    frame_counter = i
    frame_samples = sam.run(1, active_slots=[6])
    samples_list.append(frame_samples[0])

samples = np.array(samples_list)

print(f"Generated {len(samples)} frames")
print(f"Output range: L=[{samples[:, 0].min()}, {samples[:, 0].max()}], R=[{samples[:, 1].min()}, {samples[:, 1].max()}]")
print(f"SRAM reads: {len(sram_reads)}")
print(f"SRAM writes: {len(sram_writes)}")

# Check MIX state after PC05's WXY+WSP
print(f"\nFinal MIX state: mix_l={sam.state.mix_l}, mix_r={sam.state.mix_r}")

# Verify muting behavior
if sam.state.mix_l == 0 and sam.state.mix_r == 0:
    print("✓ Confirmed: Slot 6 is MUTED (mix_l=0, mix_r=0)")
else:
    print(f"✗ Unexpected: MIX not muted!")

Generated 500 frames
Output range: L=[0, 0], R=[0, 0]
SRAM reads: 2000
SRAM writes: 0

Final MIX state: mix_l=0, mix_r=0
✓ Confirmed: Slot 6 is MUTED (mix_l=0, mix_r=0)


In [7]:
# Analyze SRAM access patterns
if sram_reads:
    read_addrs = [r[1] for r in sram_reads]
    print(f"SRAM Reads: {len(sram_reads)}")
    print(f"  Reads per frame: {len(sram_reads) / NUM_FRAMES:.1f}")
    print(f"  Address range: 0x{min(read_addrs):05X} - 0x{max(read_addrs):05X}")
    print(f"  Unique addresses: {len(set(read_addrs))}")
    
    print("\nFirst 10 SRAM reads:")
    for frame, addr, val in sram_reads[:10]:
        signed_val = val if val < 2048 else val - 4096
        print(f"  Frame {frame}: addr=0x{addr:05X}, value={signed_val:+d}")
else:
    print("No SRAM reads recorded")

if sram_writes:
    write_addrs = [w[1] for w in sram_writes]
    print(f"\nSRAM Writes: {len(sram_writes)}")
    print(f"  Writes per frame: {len(sram_writes) / NUM_FRAMES:.1f}")
else:
    print("\nNo SRAM writes recorded")

SRAM Reads: 2000
  Reads per frame: 4.0
  Address range: 0x03000 - 0x03FF9
  Unique addresses: 1000

First 10 SRAM reads:
  Frame 0: addr=0x03BE0, value=+827
  Frame 0: addr=0x03BE0, value=+827
  Frame 0: addr=0x03BE1, value=+827
  Frame 0: addr=0x03BE1, value=+827
  Frame 1: addr=0x03BE8, value=+827
  Frame 1: addr=0x03BE8, value=+827
  Frame 1: addr=0x03BE9, value=+827
  Frame 1: addr=0x03BE9, value=+827
  Frame 2: addr=0x03BF0, value=+827
  Frame 2: addr=0x03BF0, value=+827

No SRAM writes recorded


In [8]:
# Final state inspection
print_state(sam.state, slot=6)

# Show D-RAM changes
print("\n" + "="*50)
print("D-RAM Word Changes (ALG 3 writes D[0,1,5,6,13,14]):")
print("="*50)
for i in [0, 1, 5, 6, 13, 14]:
    initial = dram_slot6[i]
    current = sam.state.dram[6*16 + i]
    if initial != current:
        print(f"  D[{i:2d}]: 0x{initial:05X} -> 0x{current:05X}")

SAM8905 Register State
  A = 0x4E374  (-203916)
  B = 0x248B8  (+149688)
  X = 0x47D  (+1149)
  Y = 0x008  (+8)
  PHI = 0x248  (584)
  WF = 0x1A0
  MUL = 0x0023F  (+575)

  CARRY = False
  CLEAR_RQST = True
  INT_MOD = False

  MIX_L = 0  MIX_R = 0
  L_ACC = +0  R_ACC = +0

D-RAM Slot 6:
  D[ 0] = 0x5C000  (-147456)
  D[ 1] = 0x6D280  (-77184)
  D[ 2] = 0x00600  (+1536)
  D[ 3] = 0x00080  (+128)
  D[ 4] = 0x00380  (+896)
  D[ 5] = 0x60A0D  (-128499)
  D[ 6] = 0x4E374  (-203916)
  D[ 7] = 0x52C00  (-185344)
  D[ 8] = 0x2D430  (+185392)
  D[ 9] = 0x00400  (+1024)
  D[10] = 0x7FC00  (-1024)
  D[11] = 0x00000  (+0)
  D[12] = 0x34000  (+212992)
  D[13] = 0x248B8  (+149688)
  D[14] = 0x1245C  (+74844)
  D[15] = 0x00680  (+1664)

D-RAM Word Changes (ALG 3 writes D[0,1,5,6,13,14]):
  D[ 0]: 0x5F000 -> 0x5C000
  D[ 1]: 0x7FC80 -> 0x6D280
  D[ 5]: 0x00000 -> 0x60A0D
  D[ 6]: 0x00000 -> 0x4E374
  D[13]: 0x00000 -> 0x248B8
  D[14]: 0x00000 -> 0x1245C


## Summary: ALG 3 Analysis

### Key Findings

1. **MUTED Output**
   - PC05 `WXY+WSP` updates MIX from D[7]
   - D[7] = 0x52C00 → mix_l=0, mix_r=0 (muted)
   - Despite 2 WACC instructions, output is zero

2. **SRAM Processing**
   - Multiple WXY instructions for delay buffer reads
   - WWE sequences (PC37-41, PC54-58) write back processed audio
   - This creates diffuse feedback for other slots to read

3. **Role in Reverb Architecture**
   - ALG 3 is the "diffuser" that creates late reverb texture
   - It reads from SRAM buffers and writes back all-pass filtered audio
   - Slots 7-11 (ALG 2) read from these diffused buffers
   - No direct output contribution - purely internal feedback processing

### Architecture Summary

```
ALG 0 (Slot 4): Input conditioning → SRAM write
ALG 1 (Slot 5): Diffusion → DAC output (heavy WACC)
ALG 3 (Slot 6): All-pass filter → SRAM feedback (MUTED)
ALG 2 (Slots 7-11): Delay taps → DAC output (3 WACC each)
```

The all-pass filter's muted state is intentional - it's a processing node
in the feedback network, not a direct output source.